In [1]:
import sys
import os

PROJECT_DIR = '/home/evgenshuben/Desktop/gitReps/YandexCup/'
sys.path.append(PROJECT_DIR)
os.chdir(PROJECT_DIR)

In [2]:
from omegaconf import DictConfig, OmegaConf
import hydra
from hydra import initialize, compose, initialize_config_dir
from hydra.utils import instantiate

import torch.nn as nn
import torch
import torch.nn.functional as F

In [3]:
%load_ext autoreload
%autoreload 2

from scr.train import Trainer

# Info

In [4]:
config_dir = '/home/evgenshuben/Desktop/gitReps/YandexCup/configs'
config_name = 'config'


with initialize_config_dir(config_dir=config_dir, version_base=None):
    cfg = compose(config_name=config_name)
    # cfg['dataset']['train']['train']['dataset_path']

/home/evgenshuben/miniconda3/envs/DL24/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [5]:
cfg.model.backbone_name

'hgnetv2_b4.ssld_stage2_ft_in1k'

In [6]:
trainer = Trainer(cfg)

In [7]:
for batch in trainer.train_dataloader:
    break

In [10]:
batch['anchor'].shape

torch.Size([64, 84, 50])

In [12]:
x = batch['anchor']
x.shape

torch.Size([64, 84, 50])

In [19]:
x = x.view(x.size(0), -1)
x.shape

torch.Size([64, 4200])

In [20]:
resize_layer = nn.Sequential(
    nn.Linear(84 * 50, 3 * 224 * 224),  # Преобразуем плоский вектор
    nn.ReLU(),  # Нелинейность
    nn.Unflatten(1, (3, 224, 224))  # Преобразуем в нужную размерность
)

In [22]:
resize_layer(x).shape

torch.Size([64, 3, 224, 224])

### Заметка

Оказалось, что это не очень хорошо, тк в подобном линейном слое очень много параметров около 600кк

попробую заменить это сверточными слоями

In [46]:
resize_time = nn.Linear(50, 224) 
resize_freq = nn.Linear(84, 224) 

In [47]:
for batch in trainer.train_dataloader:
    break

In [48]:
x = batch['anchor']
x.shape

torch.Size([64, 84, 50])

In [49]:
x = x.view(-1, 50)  # (batch_size * 84, 50)
x.shape

torch.Size([5376, 50])

In [50]:
x_time = resize_time(x)  # (batch_size * 84, 224)
x_time.shape

torch.Size([5376, 224])

In [51]:
x_time = x_time.view(-1, 84, 224) 
x_time.shape

torch.Size([64, 84, 224])

In [53]:
x_freq = x_time.permute(0, 2, 1)  # (batch_size, 224, 84)
x_freq = x_freq.reshape(-1, 84)
x_freq.shape

torch.Size([14336, 84])

In [55]:
x_freq = resize_freq(x_freq)
x_freq.shape

torch.Size([14336, 224])

In [56]:
x_freq = x_freq.view(-1, 224, 224)
x_freq.shape

torch.Size([64, 224, 224])

In [57]:
x_freq = x_freq.unsqueeze(1)  # (batch_size, 1, 224, 224)

In [58]:
x_freq.shape

torch.Size([64, 1, 224, 224])